In [2]:
import pandas as pd
import numpy as np
import re
from ast import *
import openai
import json
import pickle as pkl
import hashlib
from utils import *
import threading
import concurrent.futures
from config import api_key
from skill_rac_chart import skill_proficiency_level_details

# LLM Configuration

In [3]:
def get_gpt_completion(sys_msg, model="gpt-4o", temperature=0.1):
    client = openai.OpenAI(api_key=api_key,base_url="https://ai-api.analytics.gov.sg")
    response = client.chat.completions.create(
        model=model, 
        messages = sys_msg,
        response_format={"type": "json_object"},
        seed=6800,
        temperature=temperature
    )
    completion_output = literal_eval(response.choices[0].message.content)
    return completion_output

In [4]:
sys_msg = [{
    "role": "user",
    "content":"What is the colour of the sky? Reply me in JSON format {'sky_colour': 'colour'}."}]
test_result = get_gpt_completion(sys_msg=sys_msg)
test_result

{'sky_colour': 'blue'}

# Load in Data

In [5]:
round_2_output_path = "../input_data/course_skill_pl_arc_chart_full_28062024_IDD1.csv"
sfw_raw_data_path = "../input_data/Copy of Sfw_dataset-2023-10 (002).xlsx"
sfw_raw_data_sheet = "TSC_CCS_K&A"
target_sector = "Precision Engineering"

checkpoint_file_path = "./checkpoint_files/correction_checkpoint_02072024.txt"
output_file_path = "./corrected_course_skill_pl_full_02072024.csv"

In [6]:
data_df = pd.read_csv(round_2_output_path)
sfw_raw_data = pd.read_excel(sfw_raw_data_path, sheet_name=sfw_raw_data_sheet)
pe_knowledge_df = sfw_raw_data[sfw_raw_data["Sector"] == target_sector]
pe_knowledge_df = pe_knowledge_df.reset_index(drop=True)

In [8]:
pe_kb = pe_knowledge_df.groupby(['TSC_CCS Title', 'Proficiency Level'], as_index=False).agg({'Knowledge / Ability Items': ', '.join})

In [9]:
data_df["skill_pl_text"] = data_df['TSC Title/Skill'] + "-" + [str(x) for x in data_df["proficiency_level"]]
data_df["skill_pl_id"] = [generate_hash(x) for x in data_df["skill_pl_text"]]

data_df["course_text"] = data_df['Course Title'] + " |: " + data_df["About This Course"] + " | " + data_df["What You'll Learn"]
data_df['unique_text'] = data_df['course_text'] + data_df['TSC Title/Skill']
data_df['unique_id'] = data_df['unique_text'].apply(lambda x: generate_hash(x))

pe_kb["skill_pl_text"] = pe_kb["TSC_CCS Title"] + "-" + [str(x) for x in pe_kb["Proficiency Level"]]
pe_kb["skill_pl_id"] = [generate_hash(x) for x in pe_kb["skill_pl_text"]]

In [11]:
to_check_skillpl_id = data_df.skill_pl_id.unique()
reference_skillpl_id = pe_kb.skill_pl_id.unique()

In [14]:
hallucinated_skillpl_id = [x for x in to_check_skillpl_id if x not in reference_skillpl_id]

In [15]:
len(hallucinated_skillpl_id), len(reference_skillpl_id), len(to_check_skillpl_id)

(21, 292, 250)

In [16]:
affected_df = data_df[data_df["skill_pl_id"].isin(hallucinated_skillpl_id)]

In [17]:
affected_df.shape

(49, 22)

In [18]:
affected_df.head(2)

,Course Title,Course Reference Number,TP Name,TP UEN,Programme Type,About This Course,What You'll Learn,TSC Category,TSC Title/Skill,Training Provider,...,proficiency_level,reason,confidence,method (sfw / rac),Comments,skill_pl_text,skill_pl_id,course_text,unique_text,unique_id
141,(SCTP) Ã¢â‚¬â€œ Future-Ready Talent Management...,TGS-2023018975,KORN FERRY (SG) PTE. LTD.,197500776G,SSG-NON-WSQ,Our Future-Ready Talent Management Professiona...,The war for talent is heating up. Success for ...,Organisational Development,Change Management,KORN FERRY (SG) PTE. LTD.,...,2,The course focuses on acquiring basic skills i...,medium,sfw,NaN,Change Management-2,51f1c29f442600d3d73d3401b0e5dab22b4cda0505b923...,(SCTP) Ã¢â‚¬â€œ Future-Ready Talent Management...,(SCTP) Ã¢â‚¬â€œ Future-Ready Talent Management...,fc023f98c4ee8af07e3b72285988b1a77fbad93a58c924...
163,3D Printing of Advanced Polymer Products: Esse...,TGS-2019507914,NANYANG TECHNOLOGICAL UNIVERSITY,200604393R,SSG-NON-WSQ,<p style=text-align: justify;>3D printing crea...,<p style=text-align: justify;>This course intr...,Precision Manufacturing Process,Additive Manufacturing,NANYANG TECHNOLOGICAL UNIVERSITY,...,1,Course matches level 1 as it introduces basics...,high,sfw,"Based on sfw, there is no level 1 PL for Addic...",Additive Manufacturing-1,e16e55a8e97340f9f43fc663cc5f05d3e5132983568d91...,3D Printing of Advanced Polymer Products: Esse...,3D Printing of Advanced Polymer Products: Esse...,132e39226df8d46757d110b7c43b0c37bf85c7db7b8bc1...


# Correction

In [20]:
pe_kb_dic = {}
for _, row in pe_kb.iterrows():
    skill = row["TSC_CCS Title"]
    proficiency = row["Proficiency Level"]
    kna = row["Knowledge / Ability Items"]
    
    if skill not in pe_kb_dic:
        pe_kb_dic[skill] = []
        
    pe_kb_dic[skill].append({
            "proficiency": proficiency,
            "knowledge_abilities": kna
        })

In [108]:
def form_sys_msg(pe_kb_dic, crs_content, skill, skill_pl_reference_chart):
    temp_kb_dic = pe_kb_dic[skill]
    system_message = [
        {
            "role": "system",
            "content": """You are a helpful expert in the area of training courses and skills, you are helping me with associating the correct skills proficiency levels to training course contents, based on 2 sets of information:
            1. Knowledge Base that defines the knowledge and abilities that are associated with each skill at the various proficiency levels.
            2. Reference Document that defines the performance expectation for different proficiency levels.
            For each pair of course content and skill taught, identify the most appropriate proficiency level for the skill that's being taught by the training course using the Knowledge Base first. 
            Only when you need additional information, refer to the Reference Document for decision. Do not ever tag a skill with a proficiency level that's not found in the Knowledge Base.
            Give your response in JSON format like this: {{'proficiency': <>, 'reason': <>, 'confidence': <high | medium | low>}}"""
        },
        {
            "role": "user",
            "content": f"""For the training course {crs_content}, what is the most appropriate proficiency level to be tagged to the skill {skill}, based on the Knowledge Base {pe_kb_dic}, only if you need more information, refer to the Reference Document {skill_pl_reference_chart}.
            Give your response in JSON format like this: {{'proficiency': <>, 'reason': <>, 'confidence': <high | medium | low>}}"""
        }
    ]
    return system_message


def get_pl_tagging(row, id_list, pe_kb_dic, skill_pl_reference_chart):
    sys_msg = form_sys_msg(pe_kb_dic=pe_kb_dic, 
                           crs_content=row['course_text'], 
                           skill=row['TSC Title/Skill'], 
                           skill_pl_reference_chart=skill_pl_reference_chart)
    if row['unique_id'] in id_list:
        pass
    else:
        gpt_result = get_gpt_completion(sys_msg=sys_msg)
    
    return [row['unique_id'], gpt_result]


def get_result(df, max_worker, id_list, result_list, pe_kb_dic, skill_pl_reference_chart, checkpoint_filename):
    
    def process_result(row, id_list, pe_kb_dic, skill_pl_reference_chart):
        with open(checkpoint_filename, "a") as file:
            gpt_output = get_pl_tagging(row, id_list, pe_kb_dic, skill_pl_reference_chart)
            print(gpt_output[0])
            id_list.append(gpt_output[0])
            result_list.append(gpt_output[1])
            
            file.write(str(gpt_output[0]) + "\n")
        file.close()
        
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_worker) as executor:
        for _, row in df.iterrows():
            executor.submit(process_result, row, id_list, pe_kb_dic, skill_pl_reference_chart)
        
        return id_list, result_list

In [109]:
id_list = []
result_list = []

In [110]:
id_list, result_list = get_result(
    df=affected_df,
    max_worker=20,
    id_list=id_list,
    result_list=result_list,
    pe_kb_dic=pe_kb_dic,
    skill_pl_reference_chart = skill_proficiency_level_details,
    checkpoint_filename=checkpoint_file_path
)

1ab953623f84a6c0851c2fdda2a5b38751f6273cc4746721dd91a6d5e89c9d6a
734ff84d7cb512e4541b69418b675d10616d60c5ffb9ed206341d8e64da3792d
aa4aeb1de02650428adfa2898d72c675d455c2fe06c52c8e7a03720b2c5989a1
0a5f30c52fbd1e5adac6cef2ff31a7a7bc8018b263ee6cf9b131c3587cf6f527
e7af2141f436e3191b3afab24072b8145ec1a66f2f27c4e84de8411318894f48
bd79a49edf49a7b683c4b6757886d75713a219cf94ab62b0188f7f292332ce72
86290ba2c39783200377304b93b646217bfd38cf3d325437fa1e566d886f148f
0e0c3c817e158885d45639aa26591ab759c28014ed3363c73dd4a27dab7ac3d5
692b7cc06a605dc890be5e8c813fca433734ad0f62dfef19b4e6398239d6ab47
ee7b14b3ef13798242ea1f6c3c6db42379ec7894ca1f33256fe0f40fd544b849
672b1a484fa852bf9ab6f43ad00dc9beb207664f06a00d11e60d953ebf7b5b3f
1924e28ad06d39c50218bba3feb900380e3b590ddfc8d0496216496324bff2a4
bec3ae94526bc87b22965a488ac2185d768732a1df246f6d2aa873a137e11ad4
132e39226df8d46757d110b7c43b0c37bf85c7db7b8bc1438b6535245ae56792
2a4fc5de8bd20b9d550669b83fdd0e147eb3fbbfeb0befc0dbed40064b5600ef
176f2879c3725c70b9149fb08

In [111]:
len(id_list)

49

In [112]:
pickle_file_path = "./gpt_correction_output_02072024.pkl"

In [113]:
with open(pickle_file_path, "wb") as file:
    pkl.dump((id_list, result_list), file)

# Process & Check Output

In [114]:
result_df = pd.DataFrame(data={"unique_id": id_list, "proficiency_level": [x['proficiency'] for x in result_list], "reason": [x['reason'] for x in result_list], "confidence": [x['confidence'] for x in result_list]})

In [115]:
result_df.rename(columns={"proficiency_level": "proficiency_level_rac_chart", "reason": "reason_rac_chart", "confidence": "confidence_rac_chart"}, inplace=True)

In [116]:
final_df = data_df.merge(result_df, how='left', on='unique_id')

In [117]:
result_df.shape

(49, 4)

In [119]:
final_df.head(2)

,Course Title,Course Reference Number,TP Name,TP UEN,Programme Type,About This Course,What You'll Learn,TSC Category,TSC Title/Skill,Training Provider,...,method (sfw / rac),Comments,skill_pl_text,skill_pl_id,course_text,unique_text,unique_id,proficiency_level_rac_chart,reason_rac_chart,confidence_rac_chart
0,(eCornell) Rapid Ideation,TGS-2023038343,GENASHTIM,200816403N,SSG-NON-WSQ,"In this course, you will learn best practices ...",Generate ideas for new products and services\n...,Business and Organisational Management,Business Innovation,GENASHTIM,...,sfw,NaN,Business Innovation-4,016b7b236ce037862ef3ed2db7dafcc981b9d67faa5257...,"(eCornell) Rapid Ideation |: In this course, ...","(eCornell) Rapid Ideation |: In this course, ...",be689bc4430a108510eb3465e692d655f6429e9aa80336...,NaN,NaN,NaN
1,(eCornell) Rapid Ideation,TGS-2023038343,GENASHTIM,200816403N,SSG-NON-WSQ,"In this course, you will learn best practices ...",Generate ideas for new products and services\n...,Productivity and Innovation,Innovation Management,GENASHTIM,...,sfw,NaN,Innovation Management-4,1e9c2e630a224cf763e4dcf12222df558d7dc7f5e26cd6...,"(eCornell) Rapid Ideation |: In this course, ...","(eCornell) Rapid Ideation |: In this course, ...",89bf960764887f83a72e7d45aded969f317a04fcea72f1...,NaN,NaN,NaN


In [120]:
sub_final_df = final_df[~final_df.proficiency_level_rac_chart.isnull()]
sub_final_df["skill_pl_text_corrected"] = sub_final_df['TSC Title/Skill'] + "-" + [str(int(x)) for x in sub_final_df["proficiency_level_rac_chart"]]
sub_final_df["skill_pl_id_corrected"] = [generate_hash(x) for x in sub_final_df["skill_pl_text_corrected"]]

/tmp/ipykernel_1374/1721644357.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_final_df["skill_pl_text_corrected"] = sub_final_df['TSC Title/Skill'] + "-" + [str(int(x)) for x in sub_final_df["proficiency_level_rac_chart"]]
/tmp/ipykernel_1374/1721644357.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_final_df["skill_pl_id_corrected"] = [generate_hash(x) for x in sub_final_df["skill_pl_text_corrected"]]


In [121]:
corrected_skillpl_id = sub_final_df.skill_pl_id_corrected.unique()

In [134]:
if len([x for x in corrected_skillpl_id if x not in reference_skillpl_id]) == 0:
    final_df.drop(columns=['course_text','unique_text','unique_id'], inplace=True)
    final_df.to_csv(output_file_path, index=False)
    print("File saved.")
else:
    print("Still having hallucination issues.")

File saved.


In [135]:
final_df.shape

(6950, 22)